In [ ]:
pip install -q pandas numpy matplotlib seaborn xgboost scikit-learn optuna imblearn umap-learn lightgbm imbalanced-learn boto3 sagemaker


In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score
import optuna
from imblearn.over_sampling import SMOTE
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.calibration import LabelEncoder
from umap import UMAP
import optuna
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import pandas as pd
import joblib
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split 
import optuna
from imblearn.over_sampling import SMOTE 
import numpy as 
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score 
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from xgboost import XGBClassifier


DAY_ID_ENCR;CUST_LEVEL;GENDER_ENCR;R_AGE_Y;INSTALMENT_IND;INSTAL_CNT;PRODUCT_NAME;MANUFACTURER_NAME_EN;
OPERATING_SYSTEM;HANDSET_FEATURE_CAT_DESC;MOVING_AVERAGE_PRICE_AMT_ENCR;SELLING_PRICE_AMT_ENCR;UPFRONT_PYM_AMT_ENCR;monthly_fee_ENCR;
TARIFF_LEVEL;CHANNEL_CLASS;channel_group;ADDRESS_COUNTY_ENCR;OUTLET_COUNTY_ENCR;FRAUD_STATUS_6MONTH;NO_PAY_AFTER_12_MONTH


In [81]:
def preprocess_data(df):
    df_cp = df.copy(deep=True)

    #rename cols to lower case
    df_cp.columns = df_cp.columns.str.lower()

    #drop unnecesarry string cols
    #convert to day time
    df_cp.day_id_encr = pd.to_datetime(df_cp.day_id_encr, format='%Y. %m. %d.', errors='coerce')
    df_cp['year'] = df_cp['day_id_encr'].dt.year
    df_cp['month'] = df_cp['day_id_encr'].dt.month
    df_cp['day'] = df_cp['day_id_encr'].dt.day
    df_cp = df_cp.drop(columns=['day_id_encr'])
    # Type Conversion and filling NaNs
    numeric_object_cols = df_cp.select_dtypes(include=np.float64)
    for col in numeric_object_cols:
        df_cp[col] = pd.to_numeric(df_cp[col], errors='coerce')
        if df_cp[col].isnull().any():
            median_val = df_cp[col].median()
            df_cp[col] = df_cp[col].fillna(median_val)

    #filling up the age column
    df_cp.r_age_y = pd.to_numeric(df_cp.r_age_y, errors='coerce')

    # Imputation
    categorical_cols = df_cp.select_dtypes(include=['object'])
    for col in categorical_cols:
        if col in df_cp.columns:
            if df_cp[col].isnull().any():
                 df_cp[col] = df_cp[col].fillna('Unknown')
   
    #Boolean convertion
    df_cp.fraud_status_6month = df_cp.fraud_status_6month.apply(lambda x: 1 if x == 'Y' else 0) 
    df_cp.instalment_ind = df_cp.instalment_ind.apply(lambda x: 1 if x =='Y' else 0)

    return df_cp

def apply_log_transform(df, cols_to_transform):
    df_transformed = df.copy()

    for col in cols_to_transform:
        df_transformed[col] = np.log1p(df_transformed[col])

    return df_transformed

# def fit_umap_with_base_params(X_train, n_components=2, n_neighbors=15, min_dist=0.1, metric='euclidean'):
    
#     umap_model = UMAP(
#         n_components=n_components,
#         n_neighbors=n_neighbors,
#         min_dist=min_dist,
#         metric=metric,
#         random_state=DEFAULT_RANDOM_STATE,
#         transform_seed=DEFAULT_RANDOM_STATE
#     )
    
#     X_train_embeddings = umap_model.fit_transform(X_train)
    
#     return X_train_embeddings

def save_model_to_s3(model,s3_bucket,s3_key):

    local_model_filename = "model.xgb"  
    local_tarball_name = "model.tar.gz"   

    try:
       
        model.save_model(local_model_filename)

        
        with tarfile.open(local_tarball_name, "w:gz") as tar:
    
            tar.add(local_model_filename)

        s3_client = boto3.client("s3")
        s3_client.upload_file(local_tarball_name, s3_bucket, s3_key)
        
        model_s3_uri = f"s3://{s3_bucket}/{s3_key}"
        print(f"Model successfully packaged and uploaded to: {model_s3_uri}")
        return model_s3_uri

    except Exception as e:
        print(f"Error during model packaging or S3 upload: {e}")
        return None

    finally:
        if os.path.exists(local_model_filename):
            os.remove(local_model_filename)
        if os.path.exists(local_tarball_name):
            os.remove(local_tarball_name)

In [195]:

def save_optuna_study(study, filename, save_dir='optuna_studies'):
    """Saves an Optuna study object to a file using joblib."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")
        
    filepath = os.path.join(save_dir, filename)
    try:
        joblib.dump(study, filepath)
        print(f"Successfully saved Optuna study to: {filepath}")
    except Exception as e:
        print(f"Error saving Optuna study to {filepath}: {e}")

def load_optuna_study(filename):
    return joblib.load(filename)

In [ ]:

data_path = '../../../dataset.csv' 
df = pd.read_csv(data_path,decimal=',', delimiter=';', quoting=csv.QUOTE_NONE, encoding='utf-8-sig',encoding_errors='ignore', date_parser=lambda x: pd.to_datetime(x, format='%Y.%m.%d.'), na_values='?')
print(df.info())


In [84]:
#Preprocessing data columns in case of local run
df_processed = preprocess_data(df)

In [ ]:


df_processed.fillna(0,inplace=True)
numeric_cols_for_eda = df_processed.select_dtypes(include=np.float64).columns 
cols_to_remove_from_numeric = ['r_age_y', 'instal_cnt','gender_encr']
numeric_cols_for_eda = [col for col in numeric_cols_for_eda if col not in cols_to_remove_from_numeric]
numeric_cols_for_eda_df = df_processed[numeric_cols_for_eda]
correlation_matrix = numeric_cols_for_eda_df.corr()

plt.figure(figsize=(14, 14))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Numeric Features')
plt.show()
skewness = numeric_cols_for_eda_df.skew()

print("Skewness of numeric features:")
print(skewness)

for col in numeric_cols_for_eda_df.columns:
        plt.figure(figsize=(8, 6))
        sns.histplot(df_processed[col], kde=True)
        skew_val = skewness.get(col, float('nan'))
        plt.title(f'Distribution of {col} (Skewness: {skew_val:.2f})')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()

skew_threshold = 1

skewed_features_identified_from_eda = skewness[skewness > skew_threshold].index.tolist()


target_cols = ['fraud_status_6month', 'no_pay_after_12_month']

all_numeric_column_names = df_processed.drop(columns=target_cols, errors='ignore') \
                                   .select_dtypes(include=np.float64).columns.tolist()

cols_to_remove_from_numeric = ['r_age_y', 'instal_cnt','gender_encr']
all_numeric_column_names = [col for col in all_numeric_column_names if col not in cols_to_remove_from_numeric]

pipeline_skewed_numeric_features = [
    col for col in skewed_features_identified_from_eda if col in all_numeric_column_names
]

pipeline_non_skewed_numeric_features = [
    col for col in all_numeric_column_names if col not in pipeline_skewed_numeric_features
]

text_feature_colname = 'product_name'
ordinal_feature_colnames = ['cust_level', 'handset_feature_cat_desc', 'tariff_level']

all_object_cols = df_processed.select_dtypes(include='object').columns.tolist()
ohe_feature_colnames = [
    col for col in all_object_cols
    if col != text_feature_colname and col not in ordinal_feature_colnames
]

def log1p_safe(x_series):
    x_clipped = np.maximum(x_series, -1.0 + 1e-9)
    
    return np.log1p(x_clipped)

skewed_numeric_transformer_with_scale = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('logtransform', FunctionTransformer(log1p_safe, validate=False)),
    ('scaler', StandardScaler())
])

non_skewed_numeric_transformer_with_scale = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


skewed_numeric_transformer_no_scale = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('logtransform', FunctionTransformer(np.log1p, validate=False))
])

non_skewed_numeric_transformer_no_scale = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

text_transformer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=100)

ordinal_transformer = Pipeline(steps=[
    ('ordinalencoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_ohe_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='error', sparse_output=False))
])

actual_skewed_numeric_features = [col for col in pipeline_skewed_numeric_features if col in df_processed.columns]
actual_non_skewed_numeric_features = [col for col in pipeline_non_skewed_numeric_features if col in df_processed.columns]
actual_ordinal_cols = [col for col in ordinal_feature_colnames if col in df_processed.columns]
actual_ohe_cols = [col for col in ohe_feature_colnames if col in df_processed.columns]

transformers_config_with_scaling = [
    ('skewed_num', skewed_numeric_transformer_with_scale, actual_skewed_numeric_features),
    ('non_skewed_num', non_skewed_numeric_transformer_with_scale, actual_non_skewed_numeric_features),
    ('text', text_transformer, text_feature_colname),
    ('ordinal', ordinal_transformer, actual_ordinal_cols),
    ('onehot', nominal_ohe_transformer, actual_ohe_cols)  
]

preprocessor_with_scaling = ColumnTransformer(
    transformers=transformers_config_with_scaling,
    remainder='drop',
    verbose_feature_names_out=False 
)


transformers_config_no_scaling = [
    ('skewed_num', skewed_numeric_transformer_no_scale, actual_skewed_numeric_features),
    ('non_skewed_num', non_skewed_numeric_transformer_no_scale, actual_non_skewed_numeric_features),
    ('text', text_transformer, text_feature_colname),
    ('ordinal', ordinal_transformer, actual_ordinal_cols), 
    ('onehot', nominal_ohe_transformer, actual_ohe_cols)  
]

preprocessor_no_scaling = ColumnTransformer(
    transformers=transformers_config_no_scaling,
    remainder='drop',
    verbose_feature_names_out=False
)



df_features_only = df_processed.drop(columns=target_cols, errors='ignore')


X_processed_for_logreg = preprocessor_with_scaling.fit_transform(df_features_only)

X_processed_for_others = preprocessor_no_scaling.fit_transform(df_features_only)


y_fraud_6month = df_processed['fraud_status_6month'].astype(int)
y_nopay = df_processed['no_pay_after_12_month'].astype(int)

print(f"Shape of y: {y.shape}")

In [103]:

X_train, X_val, y_train, y_val = train_test_split(
    X_processed_for_others, df_processed.fraud_status_6month, test_size=0.2, random_state=42, stratify= df_processed.fraud_status_6month
)

X_train_no_pay, X_val_no_pay, y_train_no_pay, y_val_no_pay = train_test_split(
    X_processed_for_others, df_processed.no_pay_after_12_month, test_size=0.2, random_state=42, stratify=df_processed.no_pay_after_12_month
)

In [ ]:
y_train = y_train.to_numpy()
y_train_no_pay = y_train_no_pay.to_numpy()

In [106]:


X_train_lin_Reg_fraud, X_val_lin_Reg_fraud, y_train_lin_Reg_fraud, y_val_lin_Reg_fraud = train_test_split(
    X_processed_for_logreg,
    df_processed.fraud_status_6month, 
    test_size=0.2,
    random_state=42, 
    stratify=df_processed.fraud_status_6month
)

X_train_lin_Reg_no_pay, X_val_lin_Reg_no_pay, y_train_lin_Reg_no_pay, y_val_lin_Reg_no_pay = train_test_split(
    X_processed_for_logreg, 
    df_processed.no_pay_after_12_month, 
    test_size=0.2,
    random_state=42,
    stratify=df_processed.no_pay_after_12_month)

In [192]:
smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


X_train_no_pay_smote, y_train_no_pay_smote = smote.fit_resample(X_train_no_pay, y_train_no_pay)


X_train_lin_Reg_smote, y_train_lin_Reg_smote = smote.fit_resample(X_train_lin_Reg_fraud, y_train_lin_Reg_fraud)


X_train_lin_Reg_smote_no_pay, y_train_lin_Reg_smote_no_pay = smote.fit_resample(X_train_lin_Reg_no_pay, y_train_lin_Reg_no_pay)


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x=y_fraud_6month)
plt.title('Class Distribution for fraud_status_6month')
plt.xlabel('Fraud Status (6 months)')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(8, 6))
sns.countplot(x=y_nopay)
plt.title('Class Distribution for no_pay_after_12_month')
plt.xlabel('No Pay After 12 Months')
plt.ylabel('Count')
plt.show()

print("Value counts for y_fraud_6month:")
print(y_fraud_6month.value_counts(normalize=True))
print("\nValue counts for y_no_pay_after_12_month:")
print(y_nopay.value_counts(normalize=True))


In [280]:


cv_stratified = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


def objective_xgb_optuna(trial, X_train, y_train):
    """
    Optuna objective function for XGBClassifier.
    Optimizes F1-score using cross-validation.
    Handles class imbalance using scale_pos_weight.
    """

   
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), 
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 100.0, log=True),
        'random_state': 42,
        'objective': 'binary:logistic', 
        'use_label_encoder': False, 
        'eval_metric': 'logloss' 
        }

    
    model = xgb.XGBClassifier(**xgb_params)

    
    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=cv_stratified,
        scoring='f1', 
        n_jobs=-1,
        error_score='raise'
    )

    return score.mean()


def objective_logreg_smote_configurable(trial,X_train_val,y_train_val):
    logreg_c = trial.suggest_float('C', 1e-4, 1e4, log=True)
    logreg_penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    
    if logreg_penalty == 'l1':
        logreg_solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    else:
        logreg_solver = trial.suggest_categorical('solver', ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'])

    logreg_model = LogisticRegression(
        C=logreg_c,
        penalty=logreg_penalty,
        solver=logreg_solver,
        random_state=42,
        max_iter=1000 
    )
    
    score = cross_val_score(
        logreg_model, 
        X_train_val,
        y_train_val, 
        cv=cv_stratified, 
        scoring='f1', 
        n_jobs=-1
    )
    return score.mean()

from sklearn.linear_model import SGDClassifier

def objective_sgd(trial,x_train,y_train):
    sgd_loss = trial.suggest_categorical('loss', ['hinge', 'log_loss', 'modified_huber'])
    sgd_penalty = trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet'])
    sgd_alpha = trial.suggest_float('alpha', 1e-6, 1e-1, log=True) 
    sgd_l1_ratio = 0.15 
    if sgd_penalty == 'elasticnet':
        sgd_l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
        
    sgd_learning_rate = 'optimal' 

    model = SGDClassifier(
        loss=sgd_loss,
        penalty=sgd_penalty,
        alpha=sgd_alpha,
        l1_ratio=sgd_l1_ratio,
        learning_rate=sgd_learning_rate,
        max_iter=trial.suggest_int('max_iter', 500, 3000, step=500), 
        tol=1e-3, 
        shuffle=True, 
        random_state=42,
        class_weight='balanced', 
        n_jobs=-1 
    )
    
    score = cross_val_score(model, x_train, y_train, cv=cv_stratified, scoring='f1', n_jobs=-1)    
    return score.mean()


def objective_knn_configurable(trial, x_train,y_train):
    knn_n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
    knn_weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    knn_metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
    

    knn_model = KNeighborsClassifier(
        n_neighbors=knn_n_neighbors,
        weights=knn_weights,
        metric=knn_metric,
        n_jobs=-1
    )
        
    score = cross_val_score(knn_model, X_train, y_train, cv=cv_stratified, scoring='f1', n_jobs=-1)
    return score.mean()


In [ ]:
DEFAULT_N_OPTUNA_TRIALS=50

print("\nStarting Optuna for Logistic Regression (No SMOTE)...")
study_logreg_no_smote_fraud = optuna.create_study(
    direction='maximize', 
    study_name='LogisticRegression_NoSMOTE_F1_fraud'
)
study_logreg_no_smote_no_pay = optuna.create_study(
    direction='maximize', 
    study_name='LogisticRegression_NoSMOTE_F1_no_pay'
)

study_logreg_no_smote_fraud.optimize(
    lambda trial: objective_logreg_smote_configurable(trial, X_train_lin_Reg_fraud,y_train_lin_Reg_fraud), 
    n_trials=50, 
    timeout=600
)
study_logreg_no_smote_no_pay.optimize(
    lambda trial: objective_logreg_smote_configurable(trial, X_train_lin_Reg_no_pay,y_train_lin_Reg_no_pay), 
    n_trials=50, 
    timeout=600
)
print("\nBest F1 for Logistic Regression (No SMOTE) fraud:", study_logreg_no_smote_fraud.best_value)
print("\nBest F1 for Logistic Regression (No SMOTE) no pay:", study_logreg_no_smote_no_pay.best_value)



print("\nStarting Optuna for Logistic Regression (WITH SMOTE)...")
study_logreg_smote_fraud = optuna.create_study(
    direction='maximize', 
    study_name='LogisticRegression_SMOTE_F1_fraud'
)
study_logreg_smote_no_pay = optuna.create_study(
    direction='maximize', 
    study_name='LogisticRegression_SMOTE_F1_no_pay'
)
study_logreg_smote_fraud.optimize(
    lambda trial: objective_logreg_smote_configurable(trial, X_train_lin_Reg_smote,y_train_lin_Reg_smote), 
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=700 
)
study_logreg_smote_no_pay.optimize(
    lambda trial: objective_logreg_smote_configurable(trial, X_train_lin_Reg_smote_no_pay,y_train_lin_Reg_smote_no_pay), 
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=700 
)

print("\nBest F1 for Logistic Regression (WITH SMOTE):", study_logreg_smote_fraud.best_value)
print("\nBest F1 for Logistic Regression (WITH SMOTE):", study_logreg_smote_no_pay.best_value)

In [ ]:
save_optuna_study(study_logreg_no_smote_fraud,'study_logreg_no_smote_fraud.pkl')
save_optuna_study(study_logreg_no_smote_no_pay,'study_logreg_no_smote_no_pay.pkl')
save_optuna_study(study_logreg_smote_fraud,'study_logreg_smote_fraud.pkl')
save_optuna_study(study_logreg_smote_no_pay,'study_logreg_smote_no_pay.pkl')


In [ ]:


print("\nStarting Optuna for XGBoost with Focal Loss SMOTE")
study_xgb_fraud_smote = optuna.create_study(direction='maximize', study_name='XGBoost_fraud_SMOTE')
study_xgb_fraud_smote.optimize(lambda trial: objective_xgb_optuna(trial,X_train_smote,y_train_smote), n_trials=DEFAULT_N_OPTUNA_TRIALS, timeout=700) 
study_xgb_no_pay_smote = optuna.create_study(direction='maximize', study_name='XGBoost_no_pay_SMOTE')
study_xgb_no_pay_smote.optimize(lambda trial: objective_xgb_optuna(trial,X_train_no_pay_smote,y_train_no_pay_smote), n_trials=DEFAULT_N_OPTUNA_TRIALS, timeout=700) 
print("\nBest F1 for XGBoost (Actual Focal Loss) fraud SMOTE:", study_xgb_fraud_smote.best_value)
print("\nBest F1 for XGBoost (Actual Focal Loss) no_pay SMOTE:", study_xgb_no_pay_smote.best_value)

print("\nStarting Optuna for XGBoost with Focal Loss (No SMOTE)")
study_xgb_fraud_no_smote = optuna.create_study(direction='maximize', study_name='XGBoost_fraud_NoSMOTE')
study_xgb_fraud_no_smote.optimize(lambda trial: objective_xgb_optuna(trial,X_train,y_train), n_trials=DEFAULT_N_OPTUNA_TRIALS, timeout=700) 
study_xgb_no_pay_no_smote = optuna.create_study(direction='maximize', study_name='XGBoost_no_pay_NoSMOTE')
study_xgb_no_pay_no_smote.optimize(lambda trial: objective_xgb_optuna(trial,X_train_no_pay,y_train_no_pay), n_trials=DEFAULT_N_OPTUNA_TRIALS, timeout=700) 
print("\nBest F1 for XGBoost (Actual Focal Loss) fraud (No SMOTE):", study_xgb_fraud_no_smote.best_value)
print("\nBest F1 for XGBoost (Actual Focal Loss) no_pay (No SMOTE):", study_xgb_no_pay_no_smote.best_value)

In [ ]:
save_optuna_study(study_xgb_fraud_smote,'study_xgb_fraud_smote.pkl')
save_optuna_study(study_xgb_no_pay_smote,'study_xgb_no_pay_smote.pkl')
save_optuna_study(study_xgb_fraud_no_smote,'study_xgb_fraud_no_smote.pkl')
save_optuna_study(study_xgb_no_pay_no_smote,'study_xgb_no_pay_no_smote.pkl')

In [ ]:
print("\nStarting Optuna for SGDClassifier (No SMOTE)...")
study_sgd_fraud_no_smote = optuna.create_study(direction='maximize', study_name='SGDClassifier_NoSMOTE_F1_fraud')
study_sgd_fraud_no_smote.optimize(
    lambda trial: objective_sgd(trial, X_train, y_train),
    n_trials=DEFAULT_N_OPTUNA_TRIALS,
    timeout=600
)
study_sgd_no_pay_no_smote = optuna.create_study(direction='maximize', study_name='SGDClassifier_NoSMOTE_F1_no_pay')
study_sgd_no_pay_no_smote.optimize(
    lambda trial: objective_sgd(trial, X_train_no_pay, y_train_no_pay), 
    n_trials=DEFAULT_N_OPTUNA_TRIALS,
    timeout=600
)
print("\nBest F1 for SGDClassifier (No SMOTE) fraud:", study_sgd_fraud_no_smote.best_value)
print("Best params for SGDClassifier (No SMOTE) no_pay:", study_sgd_no_pay_no_smote.best_value) 

# Optuna Study for SGDClassifier WITH SMOTE
print("\nStarting Optuna for SGDClassifier (WITH SMOTE)...")
study_sgd_fraud_smote = optuna.create_study(direction='maximize', study_name='SGDClassifier_SMOTE_F1_fraud')
study_sgd_fraud_smote.optimize(
    lambda trial: objective_sgd(trial,X_train_smote,y_train_smote),
    n_trials=DEFAULT_N_OPTUNA_TRIALS,
    timeout=600
)
study_sgd_no_pay_smote = optuna.create_study(direction='maximize', study_name='SGDClassifier_SMOTE_F1_no_pay')
study_sgd_no_pay_smote.optimize(
    lambda trial: objective_sgd(trial, X_train_no_pay_smote, y_train_no_pay_smote),
    n_trials=DEFAULT_N_OPTUNA_TRIALS,
    timeout=600
)
print("\nBest F1 for SGDClassifier (WITH SMOTE) fraud:", study_sgd_fraud_smote.best_value)
print("Best params for SGDClassifier (WITH SMOTE) no_pay:", study_sgd_no_pay_smote.best_value)

In [ ]:
save_optuna_study(study_sgd_fraud_no_smote,'study_sgd_fraud_no_smote.pkl')
save_optuna_study(study_sgd_no_pay_no_smote,'study_sgd_fraud_no_pay_no_smote.pkl')
save_optuna_study(study_sgd_fraud_smote,'study_sgd_fraud_smote.pkl')
save_optuna_study(study_sgd_no_pay_smote,'study_sgd_fraud_no_pay_smote.pkl')

In [ ]:
# Optuna Study for KNN WITHOUT SMOTE
print("\nStarting Optuna for KNN (No SMOTE)...")
study_knn_fraud_no_smote = optuna.create_study(direction='maximize', study_name='KNN_NoSMOTE_F1_fraud')
study_knn_fraud_no_smote.optimize(
    lambda trial: objective_knn_configurable(trial, X_train,y_train),
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=600
)
study_knn_no_pay_no_smote = optuna.create_study(direction='maximize', study_name='KNN_NoSMOTE_F1_no_pay')
study_knn_no_pay_no_smote.optimize(
    lambda trial: objective_knn_configurable(trial,  X_train,y_train),
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=600
)
print("\nBest F1 for KNN (No SMOTE) fraud:", study_knn_fraud_no_smote.best_value)
print("Best params for KNN (No SMOTE) no_pay:", study_knn_no_pay_no_smote.best_value)

save_optuna_study(study_knn_fraud_no_smote,'knn_fraud_no_smote.pkl')
# Optuna Study for KNN WITH SMOTE
print("\nStarting Optuna for KNN (WITH SMOTE)...")
study_knn_smote_fraud = optuna.create_study(direction='maximize', study_name='KNN_SMOTE_F1_fraud')
study_knn_smote_fraud.optimize(
    lambda trial: objective_knn_configurable(trial, X_train_smote,y_train_smote),
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=600 
)

study_knn_smote_no_pay = optuna.create_study(direction='maximize', study_name='KNN_SMOTE_F1_no_pay')
study_knn_smote_no_pay.optimize(
    lambda trial: objective_knn_configurable(trial, X_train_smote,y_train_smote),
    n_trials=DEFAULT_N_OPTUNA_TRIALS, 
    timeout=600 
)
print("\nBest F1 for KNN (WITH SMOTE):", study_knn_smote_fraud.best_value)
print("Best params for KNN (WITH SMOTE):", study_knn_smote_no_pay.best_value)

In [ ]:
# Load Logistic Regression studies
study_logreg_fraud_no_smote = load_optuna_study('./optuna_studies/study_logreg_no_smote_fraud.pkl')
study_logreg_no_pay_no_smote = load_optuna_study('./optuna_studies/study_logreg_no_smote_no_pay.pkl')
study_logreg_fraud_smote = load_optuna_study('./optuna_studies/study_logreg_smote_fraud.pkl')
study_logreg_no_pay_smote = load_optuna_study('./optuna_studies/study_logreg_smote_no_pay.pkl')
print("Logistic Regression Optuna studies reloaded from .pkl files.")

# Load XGBoost studies
study_xgb_fraud_no_smote = load_optuna_study('./optuna_studies/study_xgb_fraud_no_smote.pkl')
study_xgb_no_pay_no_smote = load_optuna_study('./optuna_studies/study_xgb_no_pay_no_smote.pkl')
study_xgb_fraud_smote = load_optuna_study('./optuna_studies/study_xgb_fraud_smote.pkl')
study_xgb_no_pay_smote = load_optuna_study('./optuna_studies/study_xgb_no_pay_smote.pkl')
print("XGBoost Optuna studies reloaded from .pkl files.")

# Load SGD Classifier studies
study_sgd_fraud_no_smote = load_optuna_study('./optuna_studies/study_sgd_fraud_no_smote.pkl')
study_sgd_no_pay_no_smote = load_optuna_study('./optuna_studies/study_sgd_fraud_no_pay_no_smote.pkl') 
study_sgd_no_pay_smote = load_optuna_study('./optuna_studies/study_sgd_fraud_no_pay_smote.pkl')
print("SGD Classifier Optuna studies reloaded from .pkl files.")

study_knn_fraud_no_smote = load_optuna_study('./optuna_studies/knn_fraud_no_smote.pkl')
# study_knn_no_pay_no_smote = load_optuna_study('knn_no_pay_no_smote.pkl')
# study_knn_smote_fraud = load_optuna_study('knn_fraud_smote.pkl') 
# study_knn_smote_no_pay = load_optuna_study('knn_no_pay_smote.pkl') 
print("KNN Optuna studies reloaded from .pkl files.")

print("\nAll specified Optuna studies have been attempted to be reloaded from .pkl files.")


In [289]:

fraud_models = [
    study_logreg_fraud_no_smote,
    study_logreg_fraud_smote,
    study_xgb_fraud_no_smote,
    study_xgb_fraud_smote,
    study_sgd_fraud_no_smote,
    study_knn_fraud_no_smote
]

no_pay_models = [
    study_logreg_no_pay_no_smote,
    study_logreg_no_pay_smote,
    study_xgb_no_pay_no_smote,
    study_xgb_no_pay_smote,
    study_sgd_no_pay_no_smote,
    study_sgd_no_pay_smote 
]

MODEL_CLASSES = {
    'LogisticRegression': LogisticRegression,
    'XGBoost': XGBClassifier,
    'SGDClassifier': SGDClassifier,
    'KNN': KNeighborsClassifier
}

def train_and_evaluate(model_cls, best_params, X_train, y_train, X_test, y_test):
    model = model_cls(**best_params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = f1_score(y_test, preds, average='weighted')     
    return model, score

def validate_and_save_best(X_train, y_train, X_test, y_test, target_list,target_name):
    best_score = -1
    best_model = None
    best_model_name = ''
    for study in target_list:
        print(f"Evaluating {study.study_name} for {target_name}")
        try:
            best_params = study.best_trial.params
            model_name = study.study_name.split(
                '_'
            )[0]
            model, score = train_and_evaluate(MODEL_CLASSES[model_name], best_params, X_train, y_train, X_test, y_test)

            print(f"{model_name} accuracy: {score:.4f}")
            if score > best_score:
                best_score = score
                best_model = model
                best_model_name = model_name
        except Exception as e:
            print(f"Failed to evaluate {model_name} for {target_name}: {e}")

    if best_model:
        save_model_to_s3(best_model,'yettel-fraud-detection-inference',f'best_model_{target_name}.tar.gz')
        print(f"Saved best model ({best_model_name}) for {target_name} with score {best_score:.4f}")


In [ ]:
print("Starting evaluation for fraud models...")
validate_and_save_best(
    X_train, y_train, 
    X_val, y_val, 
    fraud_models, 
    "fraud" 
)
print("\nStarting evaluation for no-pay models...")
validate_and_save_best(
    X_train_no_pay, y_train_no_pay, 
    X_val_no_pay, y_val_no_pay, 
    no_pay_models, 
    "no_pay"  
)
